# Évaluation d'un agent IA

## Définition des variables et logging
Les variables sont lues depuis le fichier [.env](../../.env)

In [ ]:
import mlflow

# Enable autologging with all features
mlflow.dspy.autolog(
    log_compiles=True,    # Track optimization process
    log_evals=True,       # Track evaluation results
    log_traces_from_compile=True  # Track program traces during optimization
)

# Configure MLflow tracking
mlflow.set_tracking_uri("http://localhost:5000")  # Use local MLflow server
mlflow.set_experiment("10-workflow-evaluation")

from dotenv import dotenv_values
config = dotenv_values("../../.env")

llm_model = config.get('ONLINE_LLM_MODEL')
api_key = config.get('ONLINE_LLM_API_KEY')

## Configuration du llm sur dspy

In [ ]:
import dspy

lm = dspy.LM(llm_model, api_key=api_key)
# Uncomment for local api call
#lm = dspy.LM(llm_model, api_base=api_base, track_usage=True, temperature=1.5, max_tokens=1024)

dspy.configure_cache(
    enable_disk_cache=False,
    enable_memory_cache=False,
)
dspy.configure(lm=lm)

## Définitions des signatures
Définition des différentes signatures (prompt templates) qui définissent les comportements spécifiques de l'agent IA : classification d'email, résumé, réponse à une question, réponse empathique aux plaintes. Ces signatures constituent le coeur du système d'évaluation.

In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path("../../utils").resolve()))
from enum import Enum    
from file_reader import read_file


class EMAIL_INTENT(str, Enum):
    INFORMATION = "information"
    QUESTION = "question"
    ACTION = "action"
    COMPLAINT = "complaint"


class EMAIL_TONE(str, Enum):
    NEUTRAL = "neutral"
    POSITIVE = "positive"
    NEGATIVE = "negative"


class ClassifyEmail(dspy.Signature):
    """
    Analyze an incoming email to determine:
    - the sender's intent (what they expect)
    - the emotional tone (how they express it)

    The classification must be based on the email content only.
    """

    email: str = dspy.InputField(
        desc=(
            "Raw email content, including subject and body. "
            "May contain greetings, polite formulas, or emotional expressions."
        )
    )

    intent: EMAIL_INTENT = dspy.OutputField(
        desc=(
            "Primary intent of the email.\n"
            "- information: no reply or action required\n"
            "- question: asking for information\n"
            "- action: asking something to be done\n"
            "- complaint: expressing dissatisfaction or frustration"
        )
    )

    tone: EMAIL_TONE = dspy.OutputField(
        desc=(
            "Overall emotional tone of the email.\n"
            "Determine tone based on wording, urgency, politeness, "
            "and emotional expressions."
        )
    )


class SummarizeEmail(dspy.Signature):
    """
    Produce a short, factual summary of an informational email.
    The summary must capture the key message in one or two sentences.
    """

    email: str = dspy.InputField(
        desc="Informational email content to summarize."
    )

    summary: str = dspy.OutputField(
        desc="Concise summary capturing the essential information."
    )


class GenerateReply(dspy.Signature):
    """
    Generate a clear, professional reply to an email
    that asks a question or requests an action.
    """

    email: str = dspy.InputField(
        desc="Email requiring a factual or operational response."
    )

    reply: str = dspy.OutputField(
        desc=(
            "Polite, professional reply that addresses the request clearly. "
            "Do not include unnecessary empathy unless needed."
        )
    )


class EmpatheticReply(dspy.Signature):
    """
    Generate an empathetic response to a complaint email.

    The reply should:
    - acknowledge the issue
    - recognize the frustration
    - remain calm and professional
    - indicate next steps if possible
    """

    email: str = dspy.InputField(
        desc="Complaint email expressing dissatisfaction or frustration."
    )

    reply: str = dspy.OutputField(
        desc=(
            "Empathetic and professional response acknowledging the issue "
            "and reassuring the sender."
        )
    )


class EmailAgent(dspy.Module):

    def __init__(self):
        self.classifier = dspy.Predict(ClassifyEmail)
        self.summarizer = dspy.Predict(SummarizeEmail)
        self.replier = dspy.Predict(GenerateReply)
        self.empathetic = dspy.Predict(EmpatheticReply)

    def forward(self, email: str):
        analysis = self.classifier(email=email)
        result=None
        
        match analysis.intent:
            case EMAIL_INTENT.INFORMATION:
                result = self.summarizer(email=email).summary
            case EMAIL_INTENT.QUESTION:
                result = self.replier(email=email).reply
            case EMAIL_INTENT.COMPLAINT:
                result = self.empathetic(email=email).reply
            case _:
                raise Exception(f"Case for intent {analysis.intent} not implemented yet")
        
        return dspy.Prediction(
            analysis=analysis,
            result=result
        )

## Instanciation de l'agent IA
Création de l'instance de l'agent email qui sera soumis à l'évaluation. Cet agent utilise les signatures définies précédemment pour traiter différents types d'emai

In [ ]:
assistant = EmailAgent()

## Construction du jeu de données d'évaluation

Création d'un dataset complet contenant différents types d'emails avec leurs attendus (intent, tone). Ce dataset est essentiel pour évaluer la performance de l'agent IA dans des scénarios variés.

In [ ]:
evaluation_dataset = [
    # 1. Email d'information - Tone neutre
    dspy.Example(
        email="""Subject: Mise à jour du système ce weekend

Bonjour,

Nous vous informons qu'une maintenance du système aura lieu ce samedi 25 janvier de 2h à 6h du matin. 
Durant cette période, les services seront temporairement indisponibles.

Cordialement,
L'équipe IT""",
        expected_intent="information",
        expected_tone="neutral",
        expected_output_type="summary",
        expected_summary="La maintenance du système aura lieu ce samedi 25 janvier de 2h à 6h du matin. Les services seront temporairement indisponibles pendant cette période."
    ).with_inputs("email"),
    
    # 2. Email d'information - Tone positif
    dspy.Example(
        email="""Subject: Excellentes nouvelles !

Bonjour à tous,

Je suis ravi de vous annoncer que notre projet a été sélectionné pour la phase finale !
Félicitations à toute l'équipe pour cet excellent travail.

Bien cordialement,
Marie""",
        expected_intent="information",
        expected_tone="positive",
        expected_output_type="summary",
        expected_summary="Notre projet a été sélectionné pour la phase finale. Félicitations à toute l'équipe."
    ).with_inputs("email"),
    
    # 3. Question simple - Tone neutre
    dspy.Example(
        email="""Subject: Question sur les congés

Bonjour,

Pourriez-vous m'indiquer combien de jours de congés il me reste pour cette année ?

Merci d'avance,
Pierre""",
        expected_intent="question",
        expected_tone="neutral",
        expected_output_type="reply",
        expected_reply="Vous avez actuellement 15 jours de congés restants pour cette année."
    ).with_inputs("email"),
    
    # 4. Question urgente - Tone légèrement négatif
    dspy.Example(
        email="""Subject: Urgent - Accès bloqué

Bonjour,

Je n'arrive plus à accéder à mon compte depuis ce matin. J'ai besoin de ces accès pour ma présentation de cet après-midi.
Pouvez-vous m'aider rapidement ?

Cordialement,
Sophie""",
        expected_intent="question",
        expected_tone="negative",
        expected_output_type="reply",
        expected_reply="Je vous aide immédiatement à résoudre ce problème d'accès. Pouvez-vous me préciser votre nom d'utilisateur et l'erreur que vous recevez ?"
    ).with_inputs("email"),
    
    # 5. Demande d'action - Tone neutre
    dspy.Example(
        email="""Subject: Demande de modification de planning

Bonjour,

Pourriez-vous modifier ma réunion du jeudi 30 janvier à 15h et la déplacer au vendredi 31 à 10h ?

Merci,
Thomas""",
        expected_intent="action",
        expected_tone="neutral",
        expected_output_type="reply",
        expected_reply="Bien sûr, je vais modifier votre réunion. Votre réunion du jeudi 30 janvier à 15h a été déplacée au vendredi 31 à 10h."
    ).with_inputs("email"),
    
    # 6. Demande d'action urgente - Tone positif mais pressant
    dspy.Example(
        email="""Subject: Ajout à la liste de diffusion

Bonjour,

Serait-il possible d'ajouter mon collègue jean.dupont@company.com à la liste de diffusion du projet Atlas ?
Ce serait génial si cela pouvait être fait aujourd'hui.

Merci beaucoup !
Claire""",
        expected_intent="action",
        expected_tone="positive",
        expected_output_type="reply",
        expected_reply="Je vais ajouter votre collègue jean.dupont@company.com à la liste de diffusion du projet Atlas dès maintenant."
    ).with_inputs("email"),
    
    # 7. Plainte modérée - Tone négatif
    dspy.Example(
        email="""Subject: Problème récurrent avec la connexion VPN

Bonjour,

Je vous contacte car j'ai encore eu des problèmes avec le VPN ce matin. C'est la troisième fois cette semaine.
Cela commence à être problématique pour mon travail à distance.

Cordialement,
Marc""",
        expected_intent="complaint",
        expected_tone="negative",
        expected_output_type="empathetic_reply",
        expected_reply="Je comprends votre frustration concernant ces problèmes récurrents avec le VPN. Je vais investiguer ce problème et vous contacter rapidement pour résoudre cette situation."
    ).with_inputs("email"),
    
    # 8. Plainte forte - Tone très négatif
    dspy.Example(
        email="""Subject: Inacceptable - Commande non livrée

Bonjour,

Je suis extrêmement déçu. Ma commande devait arriver il y a une semaine et je n'ai toujours rien reçu !
Je n'ai eu aucune nouvelle malgré mes deux emails précédents. Ce service est vraiment décevant.

Frustré,
Antoine""",
        expected_intent="complaint",
        expected_tone="negative",
        expected_output_type="empathetic_reply",
        expected_reply="Je suis sincèrement désolé pour l'expérience que vous avez vécue concernant votre commande non livrée. Je vais immédiatement investiguer cette situation et me assurer de trouver une solution rapide."
    ).with_inputs("email"),
    
    # 9. Information avec question implicite - Tone neutre
    dspy.Example(
        email="""Subject: Absence demain

Bonjour,

Je serai absent demain pour raisons médicales. Je reprends vendredi.
Mes dossiers urgents sont sur le drive partagé.

Cordialement,
David""",
        expected_intent="information",
        expected_tone="neutral",
        expected_output_type="summary",
        expected_summary="David sera absent demain pour des raisons médicales et reprendra vendredi. Ses dossiers urgents sont disponibles sur le drive partagé."
    ).with_inputs("email"),
    
    # 10. Plainte constructive - Tone négatif mais poli
    dspy.Example(
        email="""Subject: Retour sur la nouvelle interface

Bonjour,

Je souhaitais vous faire un retour sur la nouvelle interface déployée la semaine dernière.
Bien que le design soit agréable, elle est beaucoup moins intuitive que l'ancienne version.
Plusieurs collègues partagent mon avis et nous avons du mal à retrouver certaines fonctionnalités.

Serait-il possible d'organiser une session de formation ou de revoir certains éléments ?

Merci pour votre attention,
Isabelle""",
        expected_intent="complaint",
        expected_tone="negative",
        expected_output_type="empathetic_reply",
        expected_reply="Je vous remercie pour vos retours constructifs sur la nouvelle interface. Je vais transmettre vos commentaires à l'équipe de développement et organiser une session de formation pour faciliter l'utilisation des nouvelles fonctionnalités."
    ).with_inputs("email"),
    
    # 11. Question technique - Tone neutre
    dspy.Example(
        email="""Subject: Configuration du proxy

Bonjour,

Quels sont les paramètres à utiliser pour configurer le proxy sur mon poste de travail ?

Merci,
Lucas""",
        expected_intent="question",
        expected_tone="neutral",
        expected_output_type="reply",
        expected_reply="Pour configurer le proxy sur votre poste de travail, veuillez utiliser les paramètres suivants : adresse du proxy : proxy.company.com, port : 8080, et authentification avec vos identifiants."
    ).with_inputs("email"),
    
    # 12. Action avec deadline - Tone neutre mais urgent
    dspy.Example(
        email="""Subject: Validation document avant 17h

Bonjour,

Pourriez-vous valider le document de spécifications que je vous ai envoyé ce matin ?
J'en ai besoin pour avancer et la deadline est ce soir 17h.

Merci d'avance,
Emma""",
        expected_intent="action",
        expected_tone="neutral",
        expected_output_type="reply",
        expected_reply="Je vais valider le document de spécifications dès que possible. Je vous confirme par email une fois l'approbation effectuée."
    ).with_inputs("email"),
]

## Définition de la métrique d'évaluation

Définition de la fonction de validation qui évalue si l'agent IA a correctement identifié l'intention et le ton des emails. Cette métrique est cruciale pour quantifier les performances.

L'évaluation du workflow est incomplète, l'objectif de l'exercice est également d'évaluer le llm sur le attributs générés non déterministes, pour ce faire il faut utiliser un llm pour enrichir l'évaluation

In [ ]:
def validate_answer(example, prediction, trace=None):
    return example.expected_intent.lower() == prediction.analysis.intent \
        and example.expected_tone.lower() == prediction.analysis.tone

## Lancement de l'évaluation de l'agent IA
Exécution complète de l'évaluation avec le jeu de données défini. Cette étape permet d'obtenir les métriques finales et de mesurer la performance globale de l'agent.

In [ ]:
from dspy.evaluate import Evaluate

# Set up the evaluator, which can be re-used in your code.
evaluator = Evaluate(devset=evaluation_dataset, num_threads=1, display_progress=True, display_table=5)

# Launch evaluation.
evaluator(assistant, metric=validate_answer)